In [ ]:
# Basic notebook to fetch the mhtml of a page and add it to example data

In [1]:
import os
import urllib.parse
import uuid

from playwright.async_api import async_playwright

In [2]:
# Setup a playwright instance. We need to do this to access the dev tool functions
p = await async_playwright().__aenter__()
browser = await p.chromium.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()
client = await context.new_cdp_session(page)

In [16]:
# Go to your desired URL
desired_url = "https://uoflhealth.org/locations/gi-motility-clinic-gastroenterology/"
await page.goto(desired_url)

<Response url='https://uoflhealth.org/locations/gi-motility-clinic-gastroenterology/' request=<Request url='https://uoflhealth.org/locations/gi-motility-clinic-gastroenterology/' method='GET'>>

In [10]:
def get_file_path(url: str) -> str:
    parsed_url = urllib.parse.urlparse(url)
    domain = parsed_url.netloc
    domain = domain.replace("www.", "").replace(".com", "")
    return f"./{domain}.mhtml"

In [17]:
# Capture page content as MHTML
result = await client.send('Page.captureSnapshot', {'format': 'mhtml'})
mhtml = result['data']

# Create example values
values = {
    "id": str(uuid.uuid4()),
    "url": desired_url,
    "source": "mhtml",
    "domain": "UPDATE",
    "subdomain": "UPDATE",
    "type": "fetch",
    "goal": {},
    "evals": [
      {
        "type": "json_match",
        "expected": {}
      }
    ]
  }

# Write MHTML content to the specified file
folder_path  = f"./bananalyzer/data/{values['id']}"
os.makedirs(folder_path, exist_ok=True)
file_path = os.path.join(folder_path, "index.mhtml")
with open(file_path, 'w') as f:
    f.write(mhtml)

# Print the path to the console
print(f'Page saved as MHTML at: {file_path}')
file_path = os.path.join(folder_path, "index.mhtml")
with open(file_path, 'w') as f:
    f.write(mhtml)

# Print the path to the console
print(f'Page saved as MHTML at: {file_path}')

# Save values to JSON file
import json

json_file_path = "./bananalyzer/data/examples.json"
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

data.append(values)

with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)
    json_file.write('\n')
# print(json.dumps(data, indent=4))


Page saved as MHTML at: ./bananalyzer/data/5df3d301-09f0-4ece-946d-c24952069ab7/index.mhtml
Page saved as MHTML at: ./bananalyzer/data/5df3d301-09f0-4ece-946d-c24952069ab7/index.mhtml


In [19]:
import os
import shutil
"""
Utility to delete extra mhtml folders
"""
# Read in the examples.json and fetch all of the ids
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)
ids = [item['id'] for item in data]

# Iterate through the folders
base_folder = './bananalyzer/data/'
for folder in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder)
    if folder not in ids and os.path.isdir(folder_path):
        # If there is an id that is not present and the path is a directory, delete that folder
        shutil.rmtree(folder_path)
